In [30]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np

### Conv layer와 FC layer의 weight 수 비교

In [21]:
fc = nn.Linear(in_features=3 * 32 * 32, out_features=4)
cnn = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3)

fc_num_weight = np.prod(fc.weight.size()) + fc.bias.size()[0]
cnn_num_weight = np.prod(cnn.weight.size()) + cnn.bias.size()[0]

print("fc layer weight :", fc_num_weight)
print("cnn layer weight :", cnn_num_weight)

fc layer weight : 12292
cnn layer weight : 1792


## block 쌓기

In [28]:
class ConvBlock(nn.Module):
    def __init__(self, in_channel, out_channel, kernel_size, stride, padding):
        super().__init__()
        self.conv = nn.Conv2d(in_channels=in_channel, out_channels=out_channel, kernel_size=kernel_size,
                              stride=stride, padding=padding)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.relu = nn.ReLU(inplace=True)
        
    def forward(self, inputs):
        x = self.conv(inputs)
        x = self.relu(x)
        x = self.pool(x)
        return x

In [31]:
conv = ConvBlock(3, 64, 3, 1, "same")
temp = torch.randn(1, 3, 32, 32)
conv(temp).size()

torch.Size([1, 64, 16, 16])

## 8.4 컨볼루션 신경망 구현

In [81]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_data = datasets.MNIST(root = './data/02/',
                            train=True,
                            download=True,
                            transform=transforms.ToTensor())
test_data = datasets.MNIST(root = './data/02/',
                            train=False,
                            download=True,
                            transform=transforms.ToTensor())
train_loader = torch.utils.data.DataLoader(dataset=train_data,
                                           batch_size=128, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_data,
                                           batch_size=128, shuffle=False)

class LeNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = ConvBlock(1, 6, 5, 1, "same")
        self.conv2 = ConvBlock(6, 16, 5, 1, "valid")
        self.conv3 = nn.Conv2d(in_channels=16, out_channels=120, kernel_size=5, padding="valid")
        self.fc1 = nn.Linear(in_features=120, out_features=84)
        self.relu1 = nn.ReLU(inplace=True)
        self.fc2 = nn.Linear(84, 10)
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, inputs):
        x = self.conv1(inputs)
        x = self.conv2(x)
        x = self.conv3(x)
        x = torch.flatten(x, 1)
        x = self.relu1(self.fc1(x))
        out = self.fc2(x)
        return self.softmax(out)
    
model = LeNet().to(device)    
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

model.train()
for epoch in range(3):
    for i, (data, target) in enumerate(train_loader):
        data = data.to(device)
        target = target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if (i + 1) % 100 == 0:
            print("EPOCH {}: Train Step : {}\tLoss : {:3f}".format(epoch + 1, i + 1, loss.item()))

EPOCH 1: Train Step : 100	Loss : 1.722700
EPOCH 1: Train Step : 200	Loss : 1.583469
EPOCH 1: Train Step : 300	Loss : 1.612290
EPOCH 1: Train Step : 400	Loss : 1.635639
EPOCH 2: Train Step : 100	Loss : 1.483484
EPOCH 2: Train Step : 200	Loss : 1.503726
EPOCH 2: Train Step : 300	Loss : 1.500521
EPOCH 2: Train Step : 400	Loss : 1.503649
EPOCH 3: Train Step : 100	Loss : 1.485402
EPOCH 3: Train Step : 200	Loss : 1.462831
EPOCH 3: Train Step : 300	Loss : 1.480335
EPOCH 3: Train Step : 400	Loss : 1.469154
